In [ ]:
!pip install --upgrade fiona
!pip install --upgrade pyproj
!pip install --upgrade pyshp
!pip install --upgrade shapely
!pip install --upgrade descartes
!pip install --upgrade pandas
!pip install --upgrade folium
!pip install --upgrade geopandas

In [1]:
import geopandas as gpd
import pandas as pd
import folium

## Preparação dos dados

In [7]:
# leitura dos arquivos

dir = '/content/drive/MyDrive/Programacao/GeoPandas/Dados/Dados_Modificados/'

rj = gpd.read_file(dir + 'RJ_Municipio/RJ_Municipio.shp')
imoveis = gpd.read_file(dir + 'Imoveis/Imoveis.shp')
setores_censo = gpd.read_file(dir + 'Setor_censitario/Setor_censitario.shp')
dados_agregados = gpd.read_file(dir + 'Dados_agregados/Dados_agregados.shp')
bairros = gpd.read_file(dir + 'Bairros/Bairros.shp')

In [ ]:
crs = {'init' : 'epsg:4326'} # Projeção com Latitude e Longitude

setores_censo = setores_censo.fillna(0) # retira NaN

rj.to_crs(crs, inplace=True)
imoveis.to_crs(crs, inplace=True)
setores_censo.to_crs(crs, inplace=True)
dados_agregados.to_crs(crs, inplace=True)

In [ ]:
# Centróide RJ:

x = rj.centroid.x.iloc[0] # Longitude
y = rj.centroid.y.iloc[0] # Latitude

print(x, y)
# -43.45025729553385 -22.92318738485854

## Mapa de camada simples

In [5]:
base = folium.Map([y, x], zoom_start=11, tiles='OpenStreetMap')
base.choropleth(rj)

base

/usr/local/lib/python3.7/dist-packages/folium/folium.py:413: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  FutureWarning


In [45]:
base = folium.Map([y, x], zoom_start=11, tiles='OpenStreetMap')

geojson_rj = folium.GeoJson(rj)

popup = folium.Popup(rj['NM_MUNICIP'].iloc[0])
geojson_rj.add_child(popup)

geojson_rj.add_to(base)

base

## Mapa de várias camadas

Cada Bairro é uma camada do mapa

In [30]:
base = folium.Map([y, x], zoom_start=11, tiles='OpenStreetMap')

for i in range(len(bairros)):
    geojson_bairro = folium.GeoJson(bairros[i:i+1], name=bairros['NM_BAIRRO'][i])

    """
    No método GeoJson, podemos especificar uma função anônima que define o estilo da camada.
    se tivessemos a população de cada bairro, poderiamos pintar eles de acordo com o montante:

        folium.GeoJson(bairros[i:i+1], 
                        name=bairros['NM_BAIRRO'][i], 
                        style_function = lambda feature: {
                                'fillColor': 'green' if feature['properties']['V002'] > 50000 else 'yellow',
                                'color': 'black',
                                'weight': 1
                        }
        )
    """

    label = '{}'.format(bairros['NM_BAIRRO'][i])
    popup = folium.Popup(label)
    popup.add_to(geojson_bairro)

    geojson_bairro.add_to(base)

folium.LayerControl().add_to(base)

base.save('/content/drive/MyDrive/Programacao/GeoPandas/Mapas/RJ/Bairros.html')
base

Output hidden; open in https://colab.research.google.com to view.

## Adicionando marcadores

In [51]:
base = folium.Map(location=[y, x], zoom_start=11, tiles='OpenStreetMap')
base.choropleth(bairros,
                name='Rio de Janeiro',
                line_color='Blue',
                line_weight=2,
                fill_opacity=0
                )

amostra = dados_agregados.sample(500)

for item in amostra.itertuples(): # transoforma cada linha do DF em uma tupla
    
    localizacao = [item.Latitude, item.Longitude]
    popup = "<h4>"+str(item.Tipo)+"</h4> <h5>Bairro "+str(item.Bairro)+"</h5> <p>R$ "+str(item.Valor)+"</p>"
    icone = folium.Icon(color='red', prefix='fa', icon='fas fa-home')
    marcador = folium.Marker(location=localizacao, popup=popup, icon=icone)
    
    base.add_child(marcador)

folium.LayerControl().add_to(base)

base.save('/content/drive/MyDrive/Programacao/GeoPandas/Mapas/RJ/Imoveis.html')

base

Output hidden; open in https://colab.research.google.com to view.

## Clusterizando Marcadores

In [52]:
from folium.plugins import MarkerCluster

In [55]:
base = folium.Map(location=[y, x], zoom_start=11, tiles='OpenStreetMap')
base.choropleth(bairros,
                name='Rio de Janeiro',
                line_color='Blue',
                line_weight=2,
                fill_opacity=0
                )

cluster = MarkerCluster()
cluster.layer_name = 'Imóveis anunciados'

for item in dados_agregados.itertuples():
    localizacao = [item.Latitude, item.Longitude]
    popup = "<h4>"+str(item.Tipo)+"</h4> <h5>Bairro "+str(item.Bairro)+"</h5> <p>R$ "+str(item.Valor)+"</p>"
    icone = folium.Icon(color='red', prefix='fa', icon='fas fa-home')
    marcador = folium.Marker(location=localizacao, popup=popup, icon=icone)
    
    cluster.add_child(marcador)

base.add_child(cluster)
base.add_child(folium.LayerControl())

base.save('/content/drive/MyDrive/Programacao/GeoPandas/Mapas/RJ/Imoveis_Clusterizado.html')

base

Output hidden; open in https://colab.research.google.com to view.